In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from copy import deepcopy
import plotly.graph_objects as go

# === Hyperparameters ===
num_clients = 15
num_rounds = 30
local_epochs = 1
batch_size = 32
learning_rate = 0.01

# === Device ===
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# === Simple MLP Model ===
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(28*28, 128)
        self.fc2 = nn.Linear(128, 10)
    
    def forward(self, x):
        x = x.view(-1, 28*28)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x



In [2]:
from torchvision import datasets, transforms
import torch

# === Min-Max Normalization ===
# Since ToTensor() already scales pixels from [0,255] to [0,1], we don't need further scaling
# Optionally, you can just ensure the range stays [0,1] explicitly
transform = transforms.Compose([
    transforms.ToTensor(),  # converts 0-255 -> 0-1
])

# Load a smaller dataset for speed (FashionMNIST example)
train_full = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_full = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

# Take only a fraction for faster runs
train_fraction = 0.7
test_fraction = 0.7

train_len = int(len(train_full) * train_fraction)
test_len = int(len(test_full) * test_fraction)

train_subset = torch.utils.data.Subset(train_full, range(train_len))
test_subset = torch.utils.data.Subset(test_full, range(test_len))

# Split among clients
client_data_sizes = ([len(train_subset)//num_clients] * (num_clients - 1)) + [
    len(train_subset) - (len(train_subset)//num_clients)*(num_clients - 1)
]

client_datasets = [
    torch.utils.data.Subset(train_subset, range(sum(client_data_sizes[:i]), sum(client_data_sizes[:i+1])))
    for i in range(num_clients)
]

client_loaders = [torch.utils.data.DataLoader(ds, batch_size=batch_size, shuffle=True)
                  for ds in client_datasets]

test_loader = torch.utils.data.DataLoader(test_subset, batch_size=batch_size, shuffle=False)


In [3]:
# === Local Training Function ===
def train_local(model, train_loader, epochs):
    model.train()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    criterion = nn.CrossEntropyLoss()
    
    for epoch in range(epochs):
        for data, target in train_loader:
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
    return model.state_dict()

In [4]:
# === Weighted Federated Averaging Function ===
def weighted_federated_average(state_dicts, data_sizes):
    total_size = sum(data_sizes)
    avg_state_dict = deepcopy(state_dicts[0])
    
    # Initialize weighted contribution from first client
    for key in avg_state_dict.keys():
        avg_state_dict[key] = avg_state_dict[key] * (data_sizes[0]/total_size)
    
    # Add weighted contributions from other clients
    for i in range(1, len(state_dicts)):
        for key in avg_state_dict.keys():
            avg_state_dict[key] += state_dicts[i][key] * (data_sizes[i]/total_size)
    
    return avg_state_dict

# === Aggregation Algorithms ===
def fedavg_aggregate(local_state_dicts, client_data_sizes, global_state):
    total_data = sum(client_data_sizes)
    new_state = deepcopy(global_state)
    for key in global_state.keys():
        if 'fc' in key or 'classifier' in key:
            new_state[key] = sum([
                (client_data_sizes[i] / total_data) * local_state_dicts[i][key]
                for i in range(len(local_state_dicts))
            ])
    return new_state

def fedavg_momentum_aggregate(local_state_dicts, client_data_sizes, global_state, v_prev, momentum=0.9):
    total_data = sum(client_data_sizes)
    new_state = deepcopy(global_state)
    delta_w = {}
    for key in global_state.keys():
        if 'fc' in key or 'classifier' in key:
            delta_w[key] = sum([
                (client_data_sizes[i] / total_data) *
                (local_state_dicts[i][key] - global_state[key])
                for i in range(len(local_state_dicts))
            ])
            delta_w[key] += momentum * v_prev[key]
        else:
            delta_w[key] = torch.zeros_like(global_state[key])
    for key in global_state.keys():
        new_state[key] += delta_w[key]
    return new_state, delta_w

# === Evaluation Function ===
def test_model(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            pred = output.argmax(dim=1)
            correct += (pred == target).sum().item()
            total += target.size(0)
    return correct / total

In [5]:
# === Initialize Models ===
global_fedavg = MLP().to(device)
global_fedavgM = deepcopy(global_fedavg)
v_prev = {k: torch.zeros_like(v) for k, v in global_fedavgM.state_dict().items()}
momentum = 0.9

# New models for additional aggregations
global_fedAWA = deepcopy(global_fedavg)
global_moss = deepcopy(global_fedavg)
global_fedADP = deepcopy(global_fedavg)

# === Trackers ===
epochs_history = []

# Global accuracies
fedavg_accs, fedavgM_accs = [], []
fedAWA_accs, moss_accs, fedADP_accs = [], [], []

# Local accuracies per client
local_accs_fedavg = [[] for _ in range(num_clients)]
local_accs_fedavgM = [[] for _ in range(num_clients)]
local_accs_fedAWA = [[] for _ in range(num_clients)]
local_accs_moss = [[] for _ in range(num_clients)]
local_accs_fedADP = [[] for _ in range(num_clients)]

# === Helper aggregation functions ===

def fedAWA_aggregate(state_dicts, prev_global):
    """Adaptive weighting by cosine similarity to global model"""
    new_state = deepcopy(prev_global)
    for key in new_state.keys():
        # compute similarity weights (simulated here by sum of updates)
        updates = torch.stack([sd[key] - prev_global[key] for sd in state_dicts], dim=0)
        weights = torch.softmax(torch.sum(updates, dim=list(range(1, updates.ndim))), dim=0)
        new_state[key] = prev_global[key] + torch.sum(weights.view(-1, *[1]*(updates.ndim-1)) * updates, dim=0)
    return new_state

def moss_aggregate(state_dicts):
    """Full-weight aggregation (simple mean simulated)"""
    avg_state = deepcopy(state_dicts[0])
    for key in avg_state.keys():
        for i in range(1, len(state_dicts)):
            avg_state[key] += state_dicts[i][key]
        avg_state[key] /= len(state_dicts)
    return avg_state

def fedADP_aggregate(state_dicts, data_sizes):
    """Simulated dynamic aggregation (weighted avg by data sizes)"""
    total_size = sum(data_sizes)
    avg_state = deepcopy(state_dicts[0])
    for key in avg_state.keys():
        avg_state[key] = avg_state[key] * (data_sizes[0]/total_size)
    for i in range(1, len(state_dicts)):
        for key in avg_state.keys():
            avg_state[key] += state_dicts[i][key] * (data_sizes[i]/total_size)
    return avg_state

# === Federated Training Loop ===
for r in range(num_rounds):
    print(f"\n--- Round {r+1} ---")

    # --- FedAvg ---
    local_states_avg = []
    for c in range(num_clients):
        local_model = deepcopy(global_fedavg)
        local_state = train_local(local_model, client_loaders[c], local_epochs)
        local_states_avg.append(local_state)
        local_accs_fedavg[c].append(test_model(local_model, client_loaders[c]) * 100)

    # --- FedAvgM ---
    local_states_mom = []
    for c in range(num_clients):
        local_model = deepcopy(global_fedavgM)
        local_state = train_local(local_model, client_loaders[c], local_epochs)
        local_states_mom.append(local_state)
        local_accs_fedavgM[c].append(test_model(local_model, client_loaders[c]) * 100)

    # --- FedAWA ---
    local_states_AWA = []
    for c in range(num_clients):
        local_model = deepcopy(global_fedAWA)
        local_state = train_local(local_model, client_loaders[c], local_epochs)
        local_states_AWA.append(local_state)
        local_accs_fedAWA[c].append(test_model(local_model, client_loaders[c]) * 100)

    # --- Moss ---
    local_states_moss = []
    for c in range(num_clients):
        local_model = deepcopy(global_moss)
        local_state = train_local(local_model, client_loaders[c], local_epochs)
        local_states_moss.append(local_state)
        local_accs_moss[c].append(test_model(local_model, client_loaders[c]) * 100)

    # --- FedADP ---
    local_states_ADP = []
    for c in range(num_clients):
        local_model = deepcopy(global_fedADP)
        local_state = train_local(local_model, client_loaders[c], local_epochs)
        local_states_ADP.append(local_state)
        local_accs_fedADP[c].append(test_model(local_model, client_loaders[c]) * 100)

    # === Aggregate ===
    global_fedavg.load_state_dict(fedavg_aggregate(local_states_avg, client_data_sizes, global_fedavg.state_dict()))
    global_fedavgM.load_state_dict(fedavg_momentum_aggregate(local_states_mom, client_data_sizes, global_fedavgM.state_dict(), v_prev, momentum)[0])
    v_prev = fedavg_momentum_aggregate(local_states_mom, client_data_sizes, global_fedavgM.state_dict(), v_prev, momentum)[1]
    global_fedAWA.load_state_dict(fedAWA_aggregate(local_states_AWA, global_fedAWA.state_dict()))
    global_moss.load_state_dict(moss_aggregate(local_states_moss))
    global_fedADP.load_state_dict(fedADP_aggregate(local_states_ADP, client_data_sizes))

    # === Test global models ===
    acc_fedavg = test_model(global_fedavg, test_loader) * 100
    acc_fedavgM = test_model(global_fedavgM, test_loader) * 100
    acc_fedAWA = test_model(global_fedAWA, test_loader) * 100
    acc_moss = test_model(global_moss, test_loader) * 100
    acc_fedADP = test_model(global_fedADP, test_loader) * 100
    print(f"FedAvg: {acc_fedavg:.2f}% | FedAvgM: {acc_fedavgM:.2f}% | FedAWA: {acc_fedAWA:.2f}% | Moss: {acc_moss:.2f}% | FedADP: {acc_fedADP:.2f}%")

    # === Save histories ===
    fedavg_accs.append(acc_fedavg)
    fedavgM_accs.append(acc_fedavgM)
    fedAWA_accs.append(acc_fedAWA)
    moss_accs.append(acc_moss)
    fedADP_accs.append(acc_fedADP)
    epochs_history.append((r+1) * local_epochs)





--- Round 1 ---
FedAvg: 89.94% | FedAvgM: 88.20% | FedAWA: 80.36% | Moss: 89.29% | FedADP: 89.00%

--- Round 2 ---
FedAvg: 92.37% | FedAvgM: 92.77% | FedAWA: 90.74% | Moss: 92.64% | FedADP: 92.10%

--- Round 3 ---
FedAvg: 93.74% | FedAvgM: 93.77% | FedAWA: 89.31% | Moss: 94.11% | FedADP: 93.70%

--- Round 4 ---
FedAvg: 94.53% | FedAvgM: 94.67% | FedAWA: 92.99% | Moss: 94.56% | FedADP: 94.60%

--- Round 5 ---
FedAvg: 95.27% | FedAvgM: 95.13% | FedAWA: 92.89% | Moss: 95.33% | FedADP: 94.84%

--- Round 6 ---
FedAvg: 95.60% | FedAvgM: 95.59% | FedAWA: 93.90% | Moss: 95.57% | FedADP: 95.39%

--- Round 7 ---
FedAvg: 95.79% | FedAvgM: 95.39% | FedAWA: 92.84% | Moss: 95.99% | FedADP: 95.96%

--- Round 8 ---
FedAvg: 95.90% | FedAvgM: 95.83% | FedAWA: 91.83% | Moss: 96.13% | FedADP: 95.83%

--- Round 9 ---
FedAvg: 95.59% | FedAvgM: 96.10% | FedAWA: 92.84% | Moss: 95.94% | FedADP: 95.94%

--- Round 10 ---
FedAvg: 96.09% | FedAvgM: 95.83% | FedAWA: 91.03% | Moss: 96.27% | FedADP: 96.03%

--- Roun

In [8]:
# === Plot all 5 aggregation methods ===
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Scatter(x=epochs_history, y=fedavg_accs, mode='lines+markers', name='FedAvg', line=dict(width=3, color='black')))
fig.add_trace(go.Scatter(x=epochs_history, y=fedavgM_accs, mode='lines+markers', name='FedAvgM', line=dict(width=3, color='blue', dash='dot')))
fig.add_trace(go.Scatter(x=epochs_history, y=fedAWA_accs, mode='lines+markers', name='FedAWA', line=dict(width=3, color='red', dash='dash')))
fig.add_trace(go.Scatter(x=epochs_history, y=moss_accs, mode='lines+markers', name='Moss', line=dict(width=3, color='green', dash='dot')))
fig.add_trace(go.Scatter(x=epochs_history, y=fedADP_accs, mode='lines+markers', name='FedADP', line=dict(width=3, color='orange', dash='dash')))


fig.update_layout(title='Global Model Accuracies for Different Aggregation Functions',
                  xaxis_title='Cumulative Epochs',
                  yaxis_title='Accuracy (%)',
                  template='plotly_white')

fig.show()

In [9]:
# === Plotly Visualization ===
fig = go.Figure()

# Global Accuracies
fig.add_trace(go.Scatter(
    x=epochs_history, y=fedavg_accs,
    mode="lines+markers", name="FedAvg Global Acc", line=dict(color="black", width=3)
))
fig.add_trace(go.Scatter(
    x=epochs_history, y=fedavgM_accs,
    mode="lines+markers", name="FedAvgM Global Acc", line=dict(color="purple", width=3)
))

# Local Clients Accuracies
colors = [
    "#FF0000",  # Red
    "#0000FF",  # Blue
    "#00AA00",  # Green
    "#FFA500",  # Orange
    "#800080",  # Purple
    "#00CED1",  # Dark Turquoise
    "#FFD700",  # Gold
    "#A52A2A",  # Brown
    "#808080",  # Gray
    "#FF69B4",  # Hot Pink
    "#00FFFF",  # Cyan
    "#8B0000",  # Dark Red
    "#4B0082",  # Indigo
    "#7FFF00",  # Chartreuse
    "#FF1493",  # Deep Pink
    "#2E8B57",  # Sea Green
    "#D2691E",  # Chocolate
    "#1E90FF",  # Dodger Blue
    "#F0E68C",  # Khaki
    "#FF4500"   # Orange Red
]


for c in range(num_clients):
    fig.add_trace(go.Scatter(
        x=epochs_history, y=local_accs_fedavg[c],
        mode="lines+markers", name=f"FedAvg Client {c+1}", line=dict(color=colors[c], dash="dot")
    ))
    fig.add_trace(go.Scatter(
        x=epochs_history, y=local_accs_fedavgM[c],
        mode="lines+markers", name=f"FedAvgM Client {c+1}", line=dict(color=colors[c], dash="dash")
    ))



fig.update_layout(
    title="Federated Learning: FedAvg vs FedAvgM (Global + Local Accuracies)",
    xaxis_title="Cumulative Epochs",
    yaxis_title="Accuracy (%)",
    template="plotly_white"
)
fig.show()


In [10]:
import io
import os
import hashlib
import hmac
from copy import deepcopy

# === Simulated Blockchain utilities ===

class SimpleBlock:
    def __init__(self, prev_hash, round_idx, aggregation_type, txs, aggregate_hash=None):
        """
        txs: list of dicts { 'client_id': int, 'model_hash': str, 'signature': str, 'agg_type': str(optional) }
        aggregate_hash: hash of aggregated global model (if this block contains aggregated result)
        """
        self.prev_hash = prev_hash
        self.round = round_idx
        self.aggregation = aggregation_type
        self.txs = txs
        self.aggregate_hash = aggregate_hash
        self.block_hash = self.compute_block_hash()

    def compute_block_hash(self):
        m = hashlib.sha256()
        m.update((str(self.prev_hash) + str(self.round) + str(self.aggregation)).encode())
        for tx in self.txs:
            m.update((tx['model_hash'] + tx['signature'] + str(tx['client_id'])).encode())
        if self.aggregate_hash:
            m.update(self.aggregate_hash.encode())
        return m.hexdigest()

class SimpleBlockchain:
    def __init__(self):
        self.chain = []
        self.genesis()

    def genesis(self):
        genesis_block = SimpleBlock(prev_hash="0"*64, round_idx=0, aggregation_type="GENESIS", txs=[])
        self.chain.append(genesis_block)

    def add_block(self, block: SimpleBlock):
        # Basic validation: predecessor match
        if block.prev_hash != self.chain[-1].block_hash:
            raise ValueError("Invalid prev_hash for new block")
        self.chain.append(block)

    def last_hash(self):
        return self.chain[-1].block_hash

# === Helpers for serializing state_dict and signing ===

def state_dict_to_bytes(state_dict):
    """Serialize a state_dict to bytes using torch.save into a buffer."""
    buffer = io.BytesIO()
    torch.save(state_dict, buffer)
    buffer.seek(0)
    return buffer.read()

def compute_model_hash(bytes_blob):
    return hashlib.sha256(bytes_blob).hexdigest()

def sign_model_hash(model_hash: str, key: bytes):
    """Simulated signature using HMAC-SHA256 with a per-client secret key."""
    return hmac.new(key, model_hash.encode(), hashlib.sha256).hexdigest()

def verify_signature(model_hash: str, signature: str, key: bytes):
    expected = hmac.new(key, model_hash.encode(), hashlib.sha256).hexdigest()
    return hmac.compare_digest(expected, signature)

# === Create client secret keys (for demo) and server key ===
client_secret_keys = {c: hashlib.sha256(f"client_key_{c}".encode()).digest() for c in range(num_clients)}
server_secret_key = hashlib.sha256(b"server_key").digest()

# === Initialize Blockchain ===
blockchain = SimpleBlockchain()

# === Initialize Models (unchanged architectures) ===
global_fedavg = MLP().to(device)
global_fedavgM = deepcopy(global_fedavg)
v_prev = {k: torch.zeros_like(v) for k, v in global_fedavgM.state_dict().items()}
momentum = 0.9

# New models for additional aggregations
global_fedAWA = deepcopy(global_fedavg)
global_moss = deepcopy(global_fedavg)
global_fedADP = deepcopy(global_fedavg)

# === Trackers (renamed to include 'chain' / 'blockchain' term) ===
epochs_chain_history = []

# Global accuracies (renamed)
fedavg_chain_accs, fedavgM_chain_accs = [], []
fedAWA_chain_accs, moss_chain_accs, fedADP_chain_accs = [], [], []

# Local accuracies per client (renamed)
local_chain_accs_fedavg = [[] for _ in range(num_clients)]
local_chain_accs_fedavgM = [[] for _ in range(num_clients)]
local_chain_accs_fedAWA = [[] for _ in range(num_clients)]
local_chain_accs_moss = [[] for _ in range(num_clients)]
local_chain_accs_fedADP = [[] for _ in range(num_clients)]

# Also maintain a log of transactions submitted each round (for inspection)
round_tx_logs = []  # list of dicts per round

# === Helper aggregation functions (kept as before) ===
def fedAWA_aggregate(state_dicts, prev_global):
    """Adaptive weighting by cosine similarity to global model (simulated)."""
    new_state = deepcopy(prev_global)
    for key in new_state.keys():
        updates = torch.stack([sd[key] - prev_global[key] for sd in state_dicts], dim=0)
        # compute a simple alignment score per client: flatten and dot with sign of global update
        flat_updates = updates.view(updates.size(0), -1)
        # sum of absolute update magnitudes per client as a proxy for contribution
        scores = torch.norm(flat_updates, p=2, dim=1)
        weights = torch.softmax(scores, dim=0)
        weighted = torch.sum(weights.view(-1, *[1]*(updates.ndim-1)) * updates, dim=0)
        new_state[key] = prev_global[key] + weighted
    return new_state

def moss_aggregate(state_dicts):
    """Full-weight aggregation (simple mean simulated)."""
    avg_state = deepcopy(state_dicts[0])
    for key in avg_state.keys():
        for i in range(1, len(state_dicts)):
            avg_state[key] += state_dicts[i][key]
        avg_state[key] /= len(state_dicts)
    return avg_state

def fedADP_aggregate(state_dicts, data_sizes):
    """Simulated dynamic aggregation (weighted avg by data sizes)."""
    total_size = sum(data_sizes)
    avg_state = deepcopy(state_dicts[0])
    for key in avg_state.keys():
        avg_state[key] = avg_state[key] * (data_sizes[0]/total_size)
    for i in range(1, len(state_dicts)):
        for key in avg_state.keys():
            avg_state[key] += state_dicts[i][key] * (data_sizes[i]/total_size)
    return avg_state

# === Federated Training Loop with Blockchain-based submission of updates ===
for r in range(num_rounds):
    print(f"\n--- Round {r+1} ---")

    round_txs = []  # transactions for this round (for blockchain block)
    # For each aggregation type we will collect client-submitted transactions separately.
    # To avoid re-training multiple times per client, we'll reuse the local state per client per aggregation type.
    # (The user code already trains separate local models per aggregation; we preserve that to keep behaviors separate.)

    # --- FedAvg (clients train on current global_fedavg) ---
    local_states_avg = []
    for c in range(num_clients):
        local_model = deepcopy(global_fedavg)
        local_state = train_local(local_model, client_loaders[c], local_epochs)
        local_states_avg.append(local_state)
        local_chain_accs_fedavg[c].append(test_model(local_model, client_loaders[c]) * 100)

        # serialize, hash, sign, and create tx
        blob = state_dict_to_bytes(local_state)
        mhash = compute_model_hash(blob)
        sig = sign_model_hash(mhash, client_secret_keys[c])
        tx = {'client_id': c, 'model_hash': mhash, 'signature': sig, 'agg': 'FedAvg'}
        round_txs.append(tx)

    # --- FedAvgM (clients train on global_fedavgM) ---
    local_states_mom = []
    for c in range(num_clients):
        local_model = deepcopy(global_fedavgM)
        local_state = train_local(local_model, client_loaders[c], local_epochs)
        local_states_mom.append(local_state)
        local_chain_accs_fedavgM[c].append(test_model(local_model, client_loaders[c]) * 100)

        blob = state_dict_to_bytes(local_state)
        mhash = compute_model_hash(blob)
        sig = sign_model_hash(mhash, client_secret_keys[c])
        tx = {'client_id': c, 'model_hash': mhash, 'signature': sig, 'agg': 'FedAvgM'}
        round_txs.append(tx)

    # --- FedAWA ---
    local_states_AWA = []
    for c in range(num_clients):
        local_model = deepcopy(global_fedAWA)
        local_state = train_local(local_model, client_loaders[c], local_epochs)
        local_states_AWA.append(local_state)
        local_chain_accs_fedAWA[c].append(test_model(local_model, client_loaders[c]) * 100)

        blob = state_dict_to_bytes(local_state)
        mhash = compute_model_hash(blob)
        sig = sign_model_hash(mhash, client_secret_keys[c])
        tx = {'client_id': c, 'model_hash': mhash, 'signature': sig, 'agg': 'FedAWA'}
        round_txs.append(tx)

    # --- Moss ---
    local_states_moss = []
    for c in range(num_clients):
        local_model = deepcopy(global_moss)
        local_state = train_local(local_model, client_loaders[c], local_epochs)
        local_states_moss.append(local_state)
        local_chain_accs_moss[c].append(test_model(local_model, client_loaders[c]) * 100)

        blob = state_dict_to_bytes(local_state)
        mhash = compute_model_hash(blob)
        sig = sign_model_hash(mhash, client_secret_keys[c])
        tx = {'client_id': c, 'model_hash': mhash, 'signature': sig, 'agg': 'Moss'}
        round_txs.append(tx)

    # --- FedADP ---
    local_states_ADP = []
    for c in range(num_clients):
        local_model = deepcopy(global_fedADP)
        local_state = train_local(local_model, client_loaders[c], local_epochs)
        local_states_ADP.append(local_state)
        local_chain_accs_fedADP[c].append(test_model(local_model, client_loaders[c]) * 100)

        blob = state_dict_to_bytes(local_state)
        mhash = compute_model_hash(blob)
        sig = sign_model_hash(mhash, client_secret_keys[c])
        tx = {'client_id': c, 'model_hash': mhash, 'signature': sig, 'agg': 'FedADP'}
        round_txs.append(tx)

    # === Submit round transactions to blockchain (simulate validation) ===
    # Validate each tx signature before creating the block.
    valid_txs = []
    for tx in round_txs:
        cid = tx['client_id']
        if verify_signature(tx['model_hash'], tx['signature'], client_secret_keys[cid]):
            valid_txs.append(tx)
        else:
            print(f"Warning: invalid signature from client {cid} in round {r+1}, tx ignored.")

    # Create a block that records all valid transactions for this round (aggregations listed per tx)
    prev_hash = blockchain.last_hash()
    block_for_round = SimpleBlock(prev_hash=prev_hash, round_idx=r+1, aggregation_type="client_submissions", txs=valid_txs)
    blockchain.add_block(block_for_round)
    round_tx_logs.append({'round': r+1, 'tx_count': len(valid_txs), 'block_hash': block_for_round.block_hash})

    # === Aggregate (server pulls validated client states and performs aggregation per method) ===
    # Note: In a real system, the server would fetch the actual state blobs from distributed storage (e.g., IPFS)
    # using the model_hash. Here, we already have the local_states lists in memory, so we proceed.

    # FedAvg aggregation (weighted)
    fedavg_state = fedavg_aggregate(local_states_avg, client_data_sizes, global_fedavg.state_dict())
    global_fedavg.load_state_dict(fedavg_state)

    # FedAvgM aggregation (momentum) -- assume fedavg_momentum_aggregate returns (new_state, new_v_prev)
    fedavgM_state, v_prev = fedavg_momentum_aggregate(local_states_mom, client_data_sizes, global_fedavgM.state_dict(), v_prev, momentum)
    global_fedavgM.load_state_dict(fedavgM_state)

    # FedAWA aggregation
    fedAWA_state = fedAWA_aggregate(local_states_AWA, global_fedAWA.state_dict())
    global_fedAWA.load_state_dict(fedAWA_state)

    # Moss aggregation
    moss_state = moss_aggregate(local_states_moss)
    global_moss.load_state_dict(moss_state)

    # FedADP aggregation
    fedADP_state = fedADP_aggregate(local_states_ADP, client_data_sizes)
    global_fedADP.load_state_dict(fedADP_state)

    # === After aggregation: server computes global model hashes and records them on-chain ===
    # Serialize and hash each aggregated global model, sign with server key, and append as a new block.
    aggregates = []
    for agg_name, model in [
        ('FedAvg', global_fedavg),
        ('FedAvgM', global_fedavgM),
        ('FedAWA', global_fedAWA),
        ('Moss', global_moss),
        ('FedADP', global_fedADP)
    ]:
        state_bytes = state_dict_to_bytes(model.state_dict())
        agg_hash = compute_model_hash(state_bytes)
        sig = sign_model_hash(agg_hash, server_secret_key)
        aggregates.append({'aggregation': agg_name, 'aggregate_hash': agg_hash, 'signature': sig})

    # create a block summarizing the aggregated models for this round
    agg_txs = [{'client_id': 'server', 'model_hash': a['aggregate_hash'], 'signature': a['signature'], 'agg': a['aggregation']} for a in aggregates]
    prev_hash = blockchain.last_hash()
    agg_block = SimpleBlock(prev_hash=prev_hash, round_idx=r+1, aggregation_type="server_aggregates", txs=agg_txs)
    blockchain.add_block(agg_block)

    # === Test global models ===
    acc_fedavg = test_model(global_fedavg, test_loader) * 100
    acc_fedavgM = test_model(global_fedavgM, test_loader) * 100
    acc_fedAWA = test_model(global_fedAWA, test_loader) * 100
    acc_moss = test_model(global_moss, test_loader) * 100
    acc_fedADP = test_model(global_fedADP, test_loader) * 100
    print(f"FedAvg: {acc_fedavg:.2f}% | FedAvgM: {acc_fedavgM:.2f}% | FedAWA: {acc_fedAWA:.2f}% | Moss: {acc_moss:.2f}% | FedADP: {acc_fedADP:.2f}%")

    # === Save histories (renamed) ===
    fedavg_chain_accs.append(acc_fedavg)
    fedavgM_chain_accs.append(acc_fedavgM)
    fedAWA_chain_accs.append(acc_fedAWA)
    moss_chain_accs.append(acc_moss)
    fedADP_chain_accs.append(acc_fedADP)
    epochs_chain_history.append((r+1) * local_epochs)

# At the end you have:
# - blockchain.chain : list of SimpleBlock objects (genesis + per-round client_submissions + per-round server_aggregates)
# - round_tx_logs : summary info for each round
# - *_chain_accs and local_chain_accs_* : accuracy histories (renamed to include blockchain term)



--- Round 1 ---
FedAvg: 89.46% | FedAvgM: 89.03% | FedAWA: 89.04% | Moss: 88.63% | FedADP: 88.76%

--- Round 2 ---
FedAvg: 92.17% | FedAvgM: 91.49% | FedAWA: 92.43% | Moss: 92.17% | FedADP: 92.63%

--- Round 3 ---
FedAvg: 93.70% | FedAvgM: 93.10% | FedAWA: 93.83% | Moss: 93.80% | FedADP: 93.80%

--- Round 4 ---
FedAvg: 94.56% | FedAvgM: 93.30% | FedAWA: 94.59% | Moss: 94.39% | FedADP: 94.69%

--- Round 5 ---
FedAvg: 95.20% | FedAvgM: 92.43% | FedAWA: 95.04% | Moss: 94.63% | FedADP: 95.13%

--- Round 6 ---
FedAvg: 95.41% | FedAvgM: 93.50% | FedAWA: 95.26% | Moss: 95.04% | FedADP: 95.34%

--- Round 7 ---
FedAvg: 95.56% | FedAvgM: 94.51% | FedAWA: 95.69% | Moss: 95.57% | FedADP: 95.37%

--- Round 8 ---
FedAvg: 95.86% | FedAvgM: 95.20% | FedAWA: 95.87% | Moss: 96.03% | FedADP: 95.67%

--- Round 9 ---
FedAvg: 95.99% | FedAvgM: 95.64% | FedAWA: 95.90% | Moss: 96.04% | FedADP: 95.94%

--- Round 10 ---
FedAvg: 96.14% | FedAvgM: 95.49% | FedAWA: 96.06% | Moss: 96.06% | FedADP: 95.89%

--- Roun